In [ ]:
import json
import pandas as pd
import plotly.express as px

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/datasheet.json'
with open(file_path, 'r') as file:
    data_json = json.load(file)

table_data = next(item for item in data_json if item['type'] == 'table')
data = table_data['data']
df = pd.DataFrame(data)

In [ ]:
display(df)

,id,Item,Classificação,Duração,Genero,Pontuação,Tipo,Votos,Duração_min
0,1,Stranger Things,15,60 min,"Drama, Fantasia, Terror",9,Filme,1,60
1,2,13 Reasons Why,18,60 min,"Drama, Mistério, Suspense",8,Filme,314,60
2,3,Orange Is the New Black,18,59 min,"Comédia, Crime, Drama",8,Filme,319,59
3,4,Black Mirror,18,60 min,"Drama, Mistério, Ficção Científica",9,Filme,636,60
4,5,Money Heist,15,60 min,"Ação, Crime, Drama",8,Filme,529,60
...,...,...,...,...,...,...,...,...,...
115,116,The Wanderer,14,115 min,"Drama, Aventura",8,Filme,305,115
116,117,Reign of Fire,18,104 min,"Ação, Fantasia",7,Filme,239,104
117,118,The Silence,16,90 min,"Drama, Suspense",8,Filme,322,90
118,119,Oblivion,15,125 min,"Ação, Sci-Fi",9,Série,476,125


In [ ]:
def convert_duration_to_minutes(duration):
    if 'min' in duration:
        return sum(int(x) * 60 ** i for i, x in enumerate(reversed(duration.replace(' min', '').split(','))))
    return int(duration)

df['Duração_min'] = df['Duração'].apply(convert_duration_to_minutes)
df['Tipo'] = df['Duração_min'].apply(lambda x: 'Filme' if x <= 120 else 'Série')


In [ ]:
df_filmes = df[df['Tipo'] == 'Filme']
df_series = df[df['Tipo'] == 'Série']


In [ ]:
def distruibuicao_por_genero(df, title):
    fig = px.bar(df, x='Genero', title=f'{title} - Distribuição por Gênero')
    return fig

In [ ]:
def pontuacao_por_classificacao(df, title):
    fig = px.bar(df, x='Classificação', y='Pontuação', title=f'{title} - Média de Pontuação por Classificação')
    return fig

In [ ]:
def votos_por_genero(df, title):
    fig = px.bar(df, x='Genero', y='Votos', title=f'{title} - Votos por Gênero')
    return fig

In [ ]:
def duracao_classificacao(df, title):
    fig = px.bar(df, x='Classificação', y='Duração_min', title=f'{title} - Média de Duração por Classificação')
    return fig

In [ ]:
distruibuicao_por_genero(df_filmes, 'Filmes')

In [ ]:
pontuacao_por_classificacao(df_filmes, 'Filmes')

In [ ]:
votos_por_genero(df_filmes, 'Filmes')

In [ ]:
duracao_classificacao(df_filmes, 'Filmes')

In [ ]:
distruibuicao_por_genero(df_series, 'Séries')

In [ ]:
pontuacao_por_classificacao(df_series, 'Séries')

In [ ]:
votos_por_genero(df_series, 'Séries')

In [ ]:
duracao_classificacao(df_series, 'Séries')

In [ ]:
!pip install -q streamlit

In [ ]:
!npm install localtunnel

In [ ]:
import urllib
print("SENHA/Enpoint IP para o túnel local:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
%%writefile app.py
import streamlit as st
from aula_6 import *
st.set_page_config(layout='wide')
st.title("Filmes/Séries: 🎥")
st.sidebar.title('Filtros')
filtro_classificacao = st.sidebar.multiselect(
    'Classificação',
    df['Classificação'].unique(),
)

if filtro_classificacao:
    df = df[df['Classificação'].isin(filtro_classificacao)]

aba1, aba2, aba3 = st.tabs(['Dataset', 'Filmes', 'Series'])
with aba1:
  st.dataframe(df)
with aba2:
  coluna1, coluna2 = st.columns(2)
  with coluna1:
    st.plotly_chart(distruibuicao_por_genero(df_filmes, 'Filmes'), use_container_width=True)
    st.plotly_chart(pontuacao_por_classificacao(df_filmes, 'Filmes'), use_container_width=True)
  with coluna2:
    st.plotly_chart(votos_por_genero(df_filmes, 'Filmes'), use_container_width=True)
    st.plotly_chart(duracao_classificacao(df_filmes, 'Filmes'), use_container_width=True)
with aba3:
  coluna1, coluna2 = st.columns(2)
  with coluna1:
    st.plotly_chart(distruibuicao_por_genero(df_series, 'Séries'), use_container_width=True)
    st.plotly_chart(pontuacao_por_classificacao(df_series, 'Séries'), use_container_width=True)
  with coluna2:
    st.plotly_chart(votos_por_genero(df_series, 'Séries'), use_container_width=True)
    st.plotly_chart(duracao_classificacao(df_series, 'Séries'), use_container_width=True)

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501

npx: installed 22 in 2.58s
your url is: https://tired-spies-hear.loca.lt
